In [ ]:
import sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np
from tensorflow.keras import datasets
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.boston_housing.load_data()

In [ ]:
len(x_train)

In [ ]:
layer_sizes = (100)
alpha = 0.0001 # alpha used by L2 regularization
batch_size = 10
step_size = 0.0000001


constant_model = MLPRegressor(solver="sgd", batch_size=batch_size, learning_rate_init=step_size)
adam_model = MLPRegressor(solver="adam", batch_size=batch_size)


In [ ]:
def partial_train_loop(model: sklearn.neural_network._multilayer_perceptron.MLPRegressor, n_epochs: int, train_data:np.ndarray, train_labels:np.ndarray, test_data:np.ndarray, test_labels:np.ndarray):
    errors = []
    losses = []
    for epoch in range(n_epochs):
        model.partial_fit(train_data, train_labels)
        preds = model.predict(test_data)
        errors.append(mean_absolute_error(test_labels, preds))
        losses.append(model.loss_)


    return model, errors, losses

In [ ]:
mod, errs, loss = partial_train_loop(constant_model, 50, x_train, y_train, x_test, y_test)

## altering constant step size

In [ ]:
step_sizes = [1e-8, 1e-5, 0.01]
# passing in model causes trinaing to carry over.
# step_sizes = [1e-8, 0.1]
plt.figure('step size',figsize=[10,7])
plt.rc('font', size=12)
for step in step_sizes:
    constant_model = MLPRegressor(solver="sgd",  learning_rate_init=step_size, momentum=0)

    mod, errs, loss = partial_train_loop(constant_model, 100, x_train, y_train, x_test, y_test)
    
    plt.subplot(2, 1, 1)
    plt.ylabel('Loss')
    plt.title("Constant step size")
    plt.plot(range(len(loss)), loss, label = f"step:{step}")
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.xlabel("epochs")
    plt.ylabel('Mean Absolute Error')
    plt.plot(range(len(errs)), errs, label = f"step:{step}")
# plt.title("mae")
plt.legend()


## constant step batch size

## adam alpha

In [ ]:
step_sizes = [1e-8, 1e-5, 0.01, 0.1 ]
plt.figure(figsize=[10,7])
plt.rc('font', size=12)
for step in step_sizes:
    adam_model = MLPRegressor(solver="adam", learning_rate_init=step)

    mod, errs, loss = partial_train_loop(adam_model, 100, x_train, y_train, x_test, y_test)
    
    plt.subplot(2, 1, 1)
    plt.title("Adam")
    plt.plot(range(len(loss)), loss, label = f"step:{step}")
    plt.xlabel('epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.xlabel('epochs')
    plt.ylabel('MAE')
    plt.plot(range(len(errs)), errs, label = f"step:{step}")
plt.legend()

## adam beta 1

In [ ]:
beta1_vals = [0.1, 0.3, 0.7, 0.9, 0.99]
plt.figure('beta1',figsize=[10,7])
plt.rc('font', size=12)
for bet1 in beta1_vals:
    adam_model = MLPRegressor(solver="adam", beta_1=bet1)

    mod, errs, loss = partial_train_loop(adam_model, 50, x_train, y_train, x_test, y_test)
    
    plt.subplot(2, 1, 1)
    plt.title("Adam")
    plt.plot(range(len(loss)), loss, label = f"beta 1:{bet1}")
    plt.xlabel('epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.xlabel('epochs')
    plt.ylabel('MAE')
    plt.plot(range(len(errs)), errs, label = f"beta 1:{bet1}")

plt.legend()

## adam beta 2

In [ ]:
beta2_vals = [0.1, 0.3, 0.7, 0.9, 0.99]
plt.figure('beta2',figsize=[10,7])
plt.rc('font', size=12)
for bet2 in beta2_vals:
    adam_model = MLPRegressor(solver="adam", beta_2=bet2)

    mod, errs, loss = partial_train_loop(adam_model, 50, x_train, y_train, x_test, y_test)
    
    plt.subplot(2, 1, 1)
    plt.title("Adam")
    plt.plot(range(len(loss)), loss, label = f"beta 2:{bet2}")
    plt.xlabel('epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.xlabel('epochs')
    plt.ylabel('MAE')
    plt.plot(range(len(errs)), errs, label = f"beta 2:{bet2}")

## ideal batchsize

In [ ]:
dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(x_train, y_train)
dummy_mae = mean_absolute_error(y_test,dummy_regr.predict(x_test))

In [ ]:
batch_sizes = [12,25,51 ] # default = 200
# passing in model causes trinaing to carry over.
# step_sizes = [1e-8, 0.1]
plt.figure('Batch Size Constant Step',figsize=[10,5])
plt.rc('font', size=12)
for batch in batch_sizes:
    # constant_model = MLPRegressor(solver="sgd", batch_size=batch, momentum=0, learning_rate_init=0.1)
    adam_model = MLPRegressor(solver="adam", batch_size=batch)
    # mod, errs, loss = partial_train_loop(constant_model, 100, x_train, y_train, x_test, y_test)
    adam_mod, adam_errs, adam_loss = partial_train_loop(adam_model, 100, x_train, y_train, x_test, y_test)


    # plt.plot(range(len(errs)), errs, label = f"constant, batch = {batch}")
    plt.plot(range(len(adam_errs)), adam_errs, label = f"adam, batch = {batch}")

# plt.plot(range(len(dummy_mae)),dummy_mae, label = "dumm")
plt.axhline(y = dummy_mae, label = 'Baseline')
plt.legend()
plt.title("Varying Batch Size")
plt.ylabel("Mean Absolute Error")
plt.xlabel('epochs')


In [ ]:
batch_sizes = [12, 25, 51] # default = 200
# passing in model causes trinaing to carry over.
# step_sizes = [1e-8, 0.1]
plt.figure('Batch Size Constant Step',figsize=[10,5])
plt.rc('font', size=12)
for batch in batch_sizes:
    constant_model = MLPRegressor(solver="sgd", batch_size=batch, learning_rate_init=0.1, momentum=0)

    mod, errs, loss = partial_train_loop(constant_model, 100, x_train, y_train, x_test, y_test)
    

    # plt.subplot(2, 1, 2)
    plt.xlabel("epochs")
    plt.ylabel('Mean Absolute Error')
    plt.plot(range(len(errs)), errs, label = f"batch size:{batch}")
plt.axhline(y = dummy_mae, label = 'Baseline')


plt.legend()
plt.title('Varying Batch Size, Constant')

calc mean time taken and mean mae after 100 epochs

In [ ]:
import time
iters = 10
err_sum = 0
loss_sum = 0
curr_time = time.time_ns()
for i in range(iters):
    adam_model = MLPRegressor(solver="adam", batch_size=51)
    adam_mod, adam_errs, adam_loss = partial_train_loop(adam_model, 500, x_train, y_train, x_test, y_test)
    err_sum += min(adam_errs)
    loss_sum += min(adam_loss)
adam_time = time.time_ns()- curr_time
adam_err = err_sum/iters
adam_loss_mean = loss_sum/iters

In [ ]:
err_sum = 0
# iters = 3
curr_time = time.time_ns()
for i in range(iters):
    constant_model = MLPRegressor(solver="sgd", batch_size=51, momentum=0, learning_rate_init=0.1)
    mod, errs, loss = partial_train_loop(constant_model, 500, x_train, y_train, x_test, y_test)
    err_sum += min(errs)
    loss_sum += min(loss)
const_time = time.time_ns()- curr_time
const_err = err_sum/iters
loss_mean = loss_sum/iters

In [ ]:
dummy_mae

In [ ]:
print(f"adam time {adam_time/1e9}\nconstant time {const_time/1e9}")

In [ ]:
print(f"adam error {adam_err}\nconstant error {const_err}")
print(f"adam loss {adam_loss_mean}\nconstant loss {loss_mean}")